<h1>Quantitative Momentum Strategy</h1>

"Momentum investing" means investing in the stocks that increased in price the most.

For this project. I will be building an investing strategy that returns the 50 stocks with the 

<h3>Importing Libraries</h3>

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy.stats import percentileofscore as score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

<h3>Importing List of Stocks</h3>

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secret_keys import IEX_CLOUD_API_TOKEN

<h3>Creating a Data Frame to store our data</h3>

In [ ]:
columns = [
    "Ticker",
    "Stock Price",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile",
    "HQM Score",
    "Number of Shares to Buy"
]
df = pd.DataFrame(columns=columns)

<h3>Accessing the IEX Cloud API</h3>

The IEX Cloud API allows a batch request of 100 Tickers at a time. We need to split our list into multiple lists of 100 Tickers (or less). Here is a helper function that does this:

In [ ]:
'''
Function that splits a list (l) into multiple lists of (n) items
args:
    l: array
    n: number
returns 2D array
'''
def split_list(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [ ]:
# Split our symbol list into multiple list of 100 symbols
symbols = stocks["Ticker"]
symbol_lists = list(split_list(symbols, 100))

In order to execute a batch request with the IEX Cloud API, we need to enter the stocks in the URL separated by a comma.<br>

<b>Example</b>: [AAL, AAPL, ABBV] becomes "AAL,AAPL,AABBV"

In [ ]:
symbol_batches = [] # the array that contains the symbols separated by commas
for symbol_list in symbol_lists:
    symbol_batches.append(",".join(symbol_list))

Get the key to access the API

In [ ]:
from secret_keys import IEX_CLOUD_API_TOKEN

Do batch requests

In [ ]:
for symbol_batch in symbol_batches:
    # Build URL request
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_batch}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
    # Fetch data
    data = requests.get(url).json()
    # Extract the symbols from the batch
    for symbol in symbol_batch.split(","):
        if symbol in data:
            symbol_data = data[symbol]
            symbol_stats = symbol_data["stats"]
            # Avoid getting None values
            for key in ["year1ChangePercent", "month6ChangePercent", "month3ChangePercent", "month1ChangePercent"]:
                if symbol_stats[key] == None:
                    symbol_stats[key] = 0
            # Percentiles are temporarily filled with N/A as they are calculated later in the script
            df = df.append(pd.Series([
                symbol,
                symbol_data["price"],
                symbol_stats["year1ChangePercent"],
                "N/A",
                symbol_stats["month6ChangePercent"],
                "N/A",
                symbol_stats["month3ChangePercent"],
                "N/A",
                symbol_stats["month1ChangePercent"],
                "N/A",
                "N/A",
                "N/A"
            ], index=columns), ignore_index=True)

In [ ]:
df

<h3>Calculate the Percentile of the Time Periods</h3>

In [ ]:
time_periods = ["One-Year", "Six-Month", "Three-Month", "One-Month"]

for row in df.index:
    for time_period in time_periods:
        percentile_col = f"{time_period} Return Percentile"
        change_col = f"{time_period} Price Return"
        df.loc[row, percentile_col] = score(
            df[change_col],
            df.loc[row, change_col]
        ) / 100

df

<h3>Calculate the HQM Score for each Stock</h3>

We can judge a Stock by its <b>Quality Momentum</b> - if the momentum of the stock is stable over time or if the value increases at once. High Quality Momentum Stocks increase steadily over time while Lower Quality Stocks experience unstable changes.

The HQM Score (High Quality Momentum Score) takes the mean of the Return Percentiles over a certain period of time and is calculated as a percentage (100% being a high score and 0% being a low score)

In [ ]:
from statistics import mean

for row in df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(df.loc[row, f"{time_period} Return Percentile"])
    df.loc[row, "HQM Score"] = mean(momentum_percentiles)

df

<h3>Selecting the 50 Best Momentum Stocks</h3>

In [ ]:
# Sort the values of the Data Frame
df.sort_values("HQM Score", ascending=False, inplace=True)

# Extract the 50 first elements of the sorted Data Frame
df = df[:50]

# Reset the indices
df.reset_index(inplace=True, drop=True)

df

<h3>Calculating the number of shares to buy</h3>

Get the user's portfolio size by prompting an input box

In [ ]:
def get_portfolio_size():
    portfolio_size = input("Enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print("\nYou need to enter a number.")
        val = get_portfolio_size()
    
    return val

In [ ]:
portfolio_size = get_portfolio_size()

In [ ]:
position_size = portfolio_size / len(df.index)

for row in df.index:
    df.loc[row, "Number of Shares to Buy"] = position_size // df.loc[row, "Stock Price"]

df

<h3>Export the Data Frame to an Excel File Traders can use</h3>

Initialize the XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter("quantitative_momentum_strategy.xlsx", engine="xlsxwriter")

Pass the Data Frame to the Writer Object

In [ ]:
SHEET_NAME = "Recommended Trades"
df.to_excel(writer, SHEET_NAME, index=False)

Format the excel file with the following rules:
<ul>
    <li><b>Tickers:</b> String format</li>
    <li><b>Stock Prices:</b> \$XX.XX</li>
    <li><b>Percentages:</b> %X.X</li>
    <li><b>Number of Shares to Buy:</b> Integer</li>
</ul>

In [ ]:
# Setup the styles
BACKGROUND_COLOR = "#0A0A23"
FONT_COLOR = "#FFFFFF"
BORDER_WIDTH = 1

STRING_FORMAT = writer.book.add_format({
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

PRICE_FORMAT = writer.book.add_format({
    "num_format": "$0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

VALUE_FORMAT = writer.book.add_format({
    "num_format": "$#,##0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

NUMBER_FORMAT = writer.book.add_format({
    "num_format": "0",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

PERCENT_FORMAT = writer.book.add_format({
    "num_format": "0.0%",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

map_columns_to_format = {
    "A": [columns[0], STRING_FORMAT],
    "B": [columns[1], PRICE_FORMAT],
    "C": [columns[2], PERCENT_FORMAT],
    "D": [columns[3], PERCENT_FORMAT],
    "E": [columns[4], PERCENT_FORMAT],
    "F": [columns[5], PERCENT_FORMAT],
    "G": [columns[6], PERCENT_FORMAT],
    "H": [columns[7], PERCENT_FORMAT],
    "I": [columns[8], PERCENT_FORMAT],
    "J": [columns[9], PERCENT_FORMAT],
    "K": [columns[10], PERCENT_FORMAT],
    "L": [columns[11], NUMBER_FORMAT]
}

In [ ]:
# Format Cells
for col in map_columns_to_format:
    title, cell_format = map_columns_to_format[col]
    
    # Format Header
    writer.sheets[SHEET_NAME].write(f"{col}1", title, STRING_FORMAT)
    
    # Format Body
    writer.sheets[SHEET_NAME].set_column(f"{col}:{col}", 22, cell_format)
writer.save()

Save the writer to the Excel File

In [ ]:
writer.save()